In [1]:
import gym
import numpy as np
from envs.gridworld import GridworldEnv
from envs.windy_gridworld import WindyGridworldEnv

# Policy Evaluation

In [2]:
def evaluate_policy(env, V, pi, gamma, theta):
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            v = V[s]
            bellman_update(env, V, pi, s, gamma)
            delta = max(delta, abs(v - V[s]))
        if delta <= theta:
            break
    return V

def bellman_update(env, V, pi, s, gamma):
    v = 0
    for action, p_pi in enumerate(pi[s]): 
        transitions = env.P[s][action]
        for p, s_, r, _ in transitions:
            v += p_pi * p * (r + gamma*V[s_])
    
    V[s] = v
        
def q_greedify_policy(env, V, pi, s, gamma):
    q_max = -float('inf')
    a_max = 0
    for action, _ in enumerate(pi[s]):
        q = 0
        pi[s][action] = 0
        transitions = env.P[s][action]
        
        for p, s_, r, _ in transitions:
            q += p * (r + gamma*V[s_])
            
        if (q > q_max):
            q_max = q
            a_max = action
            
    pi[s][a_max] = 1.0
    
def improve_policy(env, V, pi, gamma):
    policy_stable = True
    for s in range(env.observation_space.n):
        old = pi[s].copy()
        q_greedify_policy(env, V, pi, s, gamma)
        if not np.array_equal(pi[s], old):
            policy_stable = False
    return pi, policy_stable

def policy_iteration(env, gamma, theta):
    V = np.zeros(env.observation_space.n)
    pi = np.ones((env.observation_space.n, env.action_space.n)) / env.action_space.n
    policy_stable = False
    while not policy_stable:
        V = evaluate_policy(env, V, pi, gamma, theta)
        pi, policy_stable = improve_policy(env, V, pi, gamma)
    return V, pi

def value_iteration(env, gamma, theta):
    V = np.zeros(env.observation_space.n)
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            v = V[s]
            bellman_optimality_update(env, V, s, gamma)
            delta = max(delta, abs(v - V[s]))
        if delta <= theta:
            break
    pi = np.ones((env.observation_space.n, env.action_space.n)) / env.action_space.n
    for s in range(env.observation_space.n):
        q_greedify_policy(env, V, pi, s, gamma)
    return V, pi

def bellman_optimality_update(env, V, s, gamma):
    v_max = -float('inf')
    for action in range(env.action_space.n):
        v = 0
        transitions = env.P[s][action]
        
        for p, s_, r, _ in transitions:
            v += p * (r + gamma*V[s_])
        
        if (v > v_max):
            v_max = v
    
    V[s] = v_max

def run_policy(env, pi, episodes):
    rewards = 0
    for episode in range(episodes):
        state = env.reset()
        done = False
        episode_rewards = 0
        while not done:
            state, reward, done, _ = env.step(np.argmax(pi[state]))
            episode_rewards += reward

        rewards += episode_rewards
    return (rewards / episodes)

# Frozen Lake

In [3]:
env_fl = gym.make('FrozenLake-v0', map_name="4x4", is_slippery=False).env
env_fl_slippery = gym.make('FrozenLake-v0', map_name="4x4", is_slippery=True).env

gamma_fl = 0.999
theta_fl = 0.001
shape_fl = (4,4)

In [4]:
V_fl = np.zeros(env_fl.observation_space.n)
V_fl_slippery = np.zeros(env_fl_slippery.observation_space.n)

pi_fl = np.ones((env_fl.observation_space.n, env_fl.action_space.n)) / env_fl.action_space.n
pi_fl_slippery = np.ones((env_fl_slippery.observation_space.n, env_fl_slippery.action_space.n)) / env_fl_slippery.action_space.n

V_fl, pi_fl = policy_iteration(env_fl, gamma_fl, theta_fl)
V_fl_slippery, pi_fl_slippery = policy_iteration(env_fl_slippery, gamma_fl, theta_fl)

In [5]:
print(V_fl.reshape(shape_fl))
print(np.argmax(pi_fl, axis=1).reshape(shape_fl))

[[0.99500999 0.996006   0.997003   0.996006  ]
 [0.996006   0.         0.998001   0.        ]
 [0.997003   0.998001   0.999      0.        ]
 [0.         0.999      1.         0.        ]]
[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]


In [6]:
print(V_fl_slippery.reshape(shape_fl))
print(np.argmax(pi_fl_slippery, axis=1).reshape(shape_fl))

[[0.77068065 0.75840701 0.7500604  0.74585316]
 [0.77452039 0.         0.49532    0.        ]
 [0.78155756 0.79163217 0.73765149 0.        ]
 [0.         0.85853354 0.92832142 0.        ]]
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]


In [7]:
episodes = 10000
print("env (pi) = {}".format(run_policy(env_fl, pi_fl, episodes)))
print("env_slippery (pi_slippery) = {}".format(run_policy(env_fl_slippery, pi_fl_slippery, episodes)))
print("env_slippery (pi) = {}".format(run_policy(env_fl_slippery, pi_fl, episodes)))

env (pi) = 1.0
env_slippery (pi_slippery) = 0.8223
env_slippery (pi) = 0.0435


In [8]:
V_fl_slippery = np.zeros(env_fl_slippery.observation_space.n)
pi_fl_slippery = np.ones((env_fl_slippery.observation_space.n, env_fl_slippery.action_space.n)) / env_fl_slippery.action_space.n

V_fl_slippery, pi_fl_slippery = value_iteration(env_fl_slippery, gamma_fl, theta_fl)

print(V_fl_slippery.reshape(shape_fl))
print(np.argmax(pi_fl_slippery, axis=1).reshape(shape_fl))

[[0.7692161  0.75689418 0.7485434  0.74434035]
 [0.773235   0.         0.49463026 0.        ]
 [0.78054185 0.79094601 0.7371104  0.        ]
 [0.         0.8580588  0.92808042 0.        ]]
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]


# Grid World

In [9]:
env_grid = GridworldEnv()

gamma_grid = 1
theta_grid = 0
shape_grid=(4, 4)

V_grid = np.zeros(env_grid.observation_space.n)
pi_grid = np.ones((env_grid.observation_space.n, env_grid.action_space.n)) / env_grid.action_space.n

In [10]:
V_grid=evaluate_policy(env_grid, V_grid, pi_grid, gamma_grid, theta_grid)
pi_grid, _ = improve_policy(env_grid, V_grid, pi_grid, gamma_grid)
print(V_grid.reshape(shape_grid))
print(np.argmax(pi_grid, axis=1).reshape(shape_grid))

[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
[[0 3 3 3]
 [0 0 2 2]
 [0 1 2 2]
 [0 1 1 0]]


In [11]:
V_grid, pi_grid = value_iteration(env_grid, gamma_grid, theta_grid)
print(V_grid.reshape(shape_grid))
print(np.argmax(pi_grid, axis=1).reshape(shape_grid))

[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]


# Cliff Walk

In [12]:
env_cliff = gym.make('CliffWalking-v0')

gamma_cliff = 0.9
theta_cliff = 0.001
shape_cliff=(4, 12)

V_cliff = np.zeros(env_cliff.observation_space.n)
pi_cliff = np.ones((env_cliff.observation_space.n, env_cliff.action_space.n)) / env_cliff.action_space.n

# Need to differentiate end transition reward to get dynamic programming to work
env_cliff.P[47][0][0]=(1.0, 47, 0, True)
env_cliff.P[47][1][0]=(1.0, 47, 0, True)
env_cliff.P[47][2][0]=(1.0, 47, 0, True)
env_cliff.P[47][3][0]=(1.0, 47, 0, True)

In [13]:
V_cliff, pi_cliff = value_iteration(env_cliff, gamma_cliff, theta_cliff)
print(V_cliff.reshape(shape_cliff))
print(np.argmax(pi_cliff, axis=1).reshape(shape_cliff))

[[-7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156  -6.12579511
  -5.6953279  -5.217031   -4.68559    -4.0951     -3.439      -2.71      ]
 [-7.45813417 -7.17570464 -6.86189404 -6.5132156  -6.12579511 -5.6953279
  -5.217031   -4.68559    -4.0951     -3.439      -2.71       -1.9       ]
 [-7.17570464 -6.86189404 -6.5132156  -6.12579511 -5.6953279  -5.217031
  -4.68559    -4.0951     -3.439      -2.71       -1.9        -1.        ]
 [-7.45813417 -7.17570464 -6.86189404 -6.5132156  -6.12579511 -5.6953279
  -5.217031   -4.68559    -4.0951     -3.439      -1.          0.        ]]
[[1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 1 0]]


In [15]:
run_policy(env_cliff, pi_cliff, 1)

-13.0

In [16]:
V_cliff, pi_cliff = policy_iteration(env_cliff, gamma_cliff, theta_cliff)
print(V_cliff.reshape(shape_cliff))
print(np.argmax(pi_cliff, axis=1).reshape(shape_cliff))

[[-7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156  -6.12579511
  -5.6953279  -5.217031   -4.68559    -4.0951     -3.439      -2.71      ]
 [-7.45813417 -7.17570464 -6.86189404 -6.5132156  -6.12579511 -5.6953279
  -5.217031   -4.68559    -4.0951     -3.439      -2.71       -1.9       ]
 [-7.17570464 -6.86189404 -6.5132156  -6.12579511 -5.6953279  -5.217031
  -4.68559    -4.0951     -3.439      -2.71       -1.9        -1.        ]
 [-7.45813417 -7.17570464 -6.86189404 -6.5132156  -6.12579511 -5.6953279
  -5.217031   -4.68559    -4.0951     -3.439      -1.          0.        ]]
[[1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 1 0]]


In [17]:
run_policy(env_cliff, pi_cliff, 1)

-13.0

# Windy Grid World

In [18]:
env_windy = WindyGridworldEnv()

gamma_windy = 0.9
theta_windy = 0.001
shape_windy=(7, 10)

V_windy = np.zeros(env_windy.observation_space.n)
pi_windy = np.ones((env_windy.observation_space.n, env_windy.action_space.n)) / env_windy.action_space.n

# Need to differentiate end state reward to get dynamic programming to work
env_windy.P[37][0][0] = (1.0, 37, 0, True)
env_windy.P[37][1][0] = (1.0, 37, 0, True)
env_windy.P[37][2][0] = (1.0, 37, 0, True)
env_windy.P[37][3][0] = (1.0, 37, 0, True)

In [19]:
V_windy, pi_windy = value_iteration(env_windy, gamma_windy, theta_windy)
print(V_windy.reshape(shape_windy))
print(np.argmax(pi_windy, axis=1).reshape(shape_windy))

[[-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -5.6953279  -5.217031   -4.68559   ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -5.6953279  -5.217031   -4.0951    ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -5.6953279  -4.68559    -3.439     ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511  0.         -4.0951     -2.71      ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -1.         -1.         -1.9       ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -1.         -1.9        -1.9        -2.71      ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -1.9
  -1.9        -1.         -1.9        -2.71      ]]
[[1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 0 1 2]
 [1 1 1 1 1 1 1 2 3 3]
 [1 1 1 1 1 1 1 2 3 0]

In [20]:
run_policy(env_windy, pi_windy, 1)

-15.0

In [21]:
V_windy, pi_windy = policy_iteration(env_windy, gamma_windy, theta_windy)
print(V_windy.reshape(shape_windy))
print(np.argmax(pi_windy, axis=1).reshape(shape_windy))

[[-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -5.6953279  -5.217031   -4.68559   ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -5.6953279  -5.217031   -4.0951    ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -5.6953279  -4.68559    -3.439     ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511  0.         -4.0951     -2.71      ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -6.12579511 -1.         -1.         -1.9       ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -6.5132156
  -1.         -1.9        -1.9        -2.71      ]
 [-7.94108868 -7.71232075 -7.45813417 -7.17570464 -6.86189404 -1.9
  -1.9        -1.         -1.9        -2.71      ]]
[[1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 0 1 2]
 [1 1 1 1 1 1 1 2 3 3]
 [1 1 1 1 1 1 1 2 3 0]

In [ ]:
run_policy(env_windy, pi_windy, 1)